# Le Meilleur Data Scientist de France 2018


## Préambule 

Label Emmaüs propose à la vente en ligne des objets rénovés ou créés par le mouvement Emmaüs. Le but de ce challenge est d’estimer le délai de vente de chaque objet. 

## Description de la compétition 

Le catalogue d'objets du Label Emmaüs est en croissance régulière. L'ajout d'un objet au catalogue depuis la réception jusqu’à sa désignation (images, descriptif, entrepôt) n'est pas automatisé et prend de plus en plus de temps. 
 
La détermination d’un prix et la rédaction d’une description ne sont pas toujours simples. Il faut un peu de temps et d’expérience pour traiter rapidement un objet. L’ajout d’un produit prend aujourd’hui 40 minutes jusqu’à la mise en ligne. Beaucoup d’objets restent longtemps sur le site avant de trouver acquéreur. Et l’augmentation du catalogue nécessite de plus en plus d’espace de stockage, qui n’est pas toujours disponible. 
 
Certains magasins vendent mieux que d'autres. Ceux qui vendent moins bien créent moins d'annonces : leur espace de stockage tend à saturer et l'activité n'est pas perçue comme assez performante pour investir. A la question "pourquoi mes produits ne se vendent pas", Label Emmaüs ne sait pas répondre (sauf par du ressenti métier). 
 
L'objectif est donc de prédire la durée entre la mise en ligne et la vente d’un objet. 
 
Les données contiennent environ 10 000 objets (avec la description, le prix, la catégorie, etc.). Le délai de vente n’est connu que pour un sous-ensemble des produits (la base d’apprentissage). 
 
Il faudra prédire la durée pour les produits de la base de tests.

La cible est catégorisée en 3 modalités: 
 
- 0 : entre 0 et 10 jours 
- 1 : entre 10 jours et 60 jours 
- 2 : plus de 60 jours
 
Vous avez 2 heures pour résoudre le challenge. Les soumissions réalisées avant et après la fin du concours ne seront pas prises en compte.

## Format de soumission

Le fichier soumis doit contenir 4 colonnes : 

- colonne 1 : id produit 
- colonne 2 : probabilité que le produit appartient à la catégorie delai_de_vente = 0 (entre 0 et 10 jours) 
- colonne 3 : probabilité que le produit appartient à la catégorie delai_de_vente = 1 (entre 11 et 60 jours) 
- colonne 4 : probabilité que le produit appartient à la catégorie delai_de_vente = 2 (plus de 60 jours) 

 
Le séparateur de colonnes est la “,” (virgule). 
Le séparateur décimal est le “.” (point). 

## Métrique d'évaluation

Log Loss

http://wiki.fast.ai/index.php/Log_Loss

# import modules and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
pd.set_option('display.max_columns', 500)

In [4]:
X_train = pd.read_csv("X_train.csv", index_col=0, error_bad_lines=False)
X_test = pd.read_csv("X_test.csv", index_col=0, error_bad_lines=False)
y_train = pd.read_csv("y_train.csv", index_col=0)

Skipping line 2168: expected 31 fields, saw 33
Skipping line 4822: expected 31 fields, saw 37
Skipping line 4859: expected 31 fields, saw 37
Skipping line 7342: expected 31 fields, saw 37



In [5]:
X_train_index = X_train.index
X_test_index = X_test.index

# exploration

In [6]:
print("Dimension X_train:", X_train.shape)
print("Dimension X_test:", X_test.shape)

('Dimension X_train:', (8880, 30))
('Dimension X_test:', (2960, 30))


In [7]:
X_train.head(3)

,nb_images,longueur_image,largeur_image,url_image,description_produit,taille,matiere,age,garantie,annee,couleur,largeur_produit,wifi,etat,longueur_produit,pointure,vintage,marque,auteur,editions,hauteur_produit,poids,prix,categorie,sous_categorie_1,sous_categorie_2,sous_categorie_3,sous_categorie_4,nom_produit,nom_magasin
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3,3458.0,2552.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Superbe petit top bustier avec explosion de co...,44,100 % polyester,NaN,NaN,NaN,Multicolore,NaN,NaN,bon état,NaN,NaN,False,NaN,NaN,NaN,NaN,200.0,4.5,mode,"tops, t-shirts, débardeurs femme",NaN,NaN,NaN,Top bustier multicolore,Emmaüs 88 Neufchateau
1,2,2486.0,2254.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,"Radio ITT Océnic Flirt, année 70\nPour déco",NaN,Plastique,NaN,NaN,NaN,Jaune,NaN,NaN,en l'état,NaN,NaN,True,ITT Océanic,NaN,NaN,NaN,1000.0,15.0,mobilier - deco,bibelots et objets déco,NaN,NaN,NaN,Radio ITT Océanic,Communauté Emmaüs Thouars (magasin Parthenay)
2,3,1536.0,1536.0,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,Veste boléro à manches courtes NÛMPH. Gris chi...,40,"Polyester, coton, laine",NaN,NaN,NaN,Gris,NaN,NaN,neuf,NaN,NaN,False,Nûmph,NaN,NaN,NaN,360.0,16.0,label selection,mode,mode femme,NaN,NaN,NaN,Label Emmaüs Chambéry


In [8]:
X_train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
nb_images,8880,NaN,NaN,NaN,3.63345,2.04857,0,2,3,5,29
longueur_image,8823,NaN,NaN,NaN,1807.82,1025.25,58,1000,1536,2448,5472
largeur_image,8823,NaN,NaN,NaN,1801.77,1101.21,64,970.5,1536,2448,5472
url_image,8823,8775,https://d1kvfoyrif6wzg.cloudfront.net/assets/i...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description_produit,8880,8836,"Relié, 48 pages, couverture usagée",6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taille,2414,33,38,402,NaN,NaN,NaN,NaN,NaN,NaN,NaN
matiere,3947,1722,Coton,144,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,120,18,4a,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
garantie,101,2,6 mois,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annee,1497,NaN,NaN,NaN,14810.1,496237,0,1979,1998,2007,1.92019e+07


In [9]:
y_train.delai_vente.value_counts()

0    3027
2    2953
1    2900
Name: delai_vente, dtype: int64

In [10]:
X_train.etat.value_counts()

bon état         4537
en l'état        1577
comme neuf       1110
reconditionné     885
neuf              747
Name: etat, dtype: int64

# feature engineering

In [27]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [12]:
feat_num = ["poids", "prix", "nb_images", "longueur_image", "largeur_image"]
feat_cat = ["garantie","couleur","wifi","etat","vintage","categorie","taille","age","sous_categorie_1","sous_categorie_2","sous_categorie_3","sous_categorie_4","nom_magasin","matiere","auteur","editions","marque"]
feat_engineered = []

In [13]:
# etat to ordinal
def ordinalEtat(string):
    if(string=="en l'état"):
        return(1)
    elif(string=="bon état"):
        return(2)
    elif(string=="reconditionné"):
        return(3)
    elif(string=="comme neuf"):
        return(4)
    elif(string=="neuf"):
        return(5)
    
X_train["ordinaletat"] = X_train["etat"].apply(lambda x: ordinalEtat(x))
X_test["ordinaletat"] = X_test["etat"].apply(lambda x: ordinalEtat(x))
feat_engineered.append("ordinaletat")

In [14]:
# dummify categorical columns

dummies_train = pd.get_dummies(X_train[feat_cat])
dummies_test = pd.get_dummies(X_test[feat_cat])

feat_cat_dummified = dummies_train.columns

X_train = pd.concat([X_train, dummies_train], axis=1)
X_train = X_train.drop(feat_cat, axis=1)

X_test = pd.concat([X_test, dummies_test], axis=1)
X_test = X_test.drop(feat_cat, axis=1)

In [15]:
# nb characters description_produit
X_train["nbChardescription_produit"] = X_train["description_produit"].apply(lambda x: len(x))
X_test["nbChardescription_produit"] = X_test["description_produit"].apply(lambda x: len(x))
feat_engineered.append("nbChardescription_produit")

In [16]:
# has largeur_produit
X_train["haslargeur_produit"] = X_train["largeur_produit"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
X_test["haslargeur_produit"] = X_test["largeur_produit"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
feat_engineered.append("haslargeur_produit")

In [17]:
# has longueur_produit
X_train["haslongueur_produit"] = X_train["longueur_produit"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
X_test["haslongueur_produit"] = X_test["longueur_produit"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
feat_engineered.append("haslongueur_produit")

In [18]:
# has pointure
X_train["haspointure"] = X_train["pointure"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
X_test["haspointure"] = X_test["pointure"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
feat_engineered.append("haspointure")

In [19]:
# has hauteur_produit
X_train["hashauteur_produit"] = X_train["hauteur_produit"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
X_test["hashauteur_produit"] = X_test["hauteur_produit"].apply(lambda x: 0 if pd.isnull(x) else 1).astype('bool')
feat_engineered.append("hashauteur_produit")

In [20]:
# nb characters nom_produit
X_train["nbCharnom_produit"] = X_train["nom_produit"].apply(lambda x: 0 if pd.isnull(x) else len(str(x)))
X_test["nbCharnom_produit"] = X_test["nom_produit"].apply(lambda x: 0 if pd.isnull(x) else len(str(x)))
feat_engineered.append("nbCharnom_produit")

In [21]:
# nom_magasin contains Emmaüs
X_train["nom_magasinContainsEmmaus"] = X_train["description_produit"].apply(lambda x: 0 if pd.isnull(x) else "Emmaüs" in x).astype('bool')
X_test["nom_magasinContainsEmmaus"] = X_test["description_produit"].apply(lambda x: 0 if pd.isnull(x) else "Emmaüs" in x).astype('bool')
feat_engineered.append("nom_magasinContainsEmmaus")

In [22]:
# has image nb_images
X_train["hasimage"] = X_train["nb_images"].apply(lambda x: 0 if x == 0 else 1).astype('bool')
X_test["hasimage"] = X_test["nb_images"].apply(lambda x: 0 if x == 0 else 1).astype('bool')
feat_engineered.append("hasimage")

# select only features used for modelling

In [23]:
features = feat_num+list(feat_cat_dummified)+feat_engineered
print(len(features))

6682


In [24]:
X_train = X_train.loc[:, features]
X_test = X_test.loc[:, features]

/home/jonathan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


# missing data imputation

In [28]:
# train
imp = SimpleImputer(strategy='median')
imp.fit(X_train)
X_train = imp.transform(X_train)

# test
X_test = imp.transform(X_test)

# model selection and hyperparameter tuning

## random forest

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss 

In [30]:
# max_depth values to experiment
max_depth_values = [5,10,20,100,200]

In [31]:
%%time

# for each possible value of max_depth: print CV logloss
for depth in max_depth_values:
    rf = RandomForestClassifier(n_estimators=50,n_jobs=-1,max_depth=depth)
    neg_log_losses = cross_val_score(rf, X_train, np.ravel(y_train), cv=5, scoring="neg_log_loss")
    log_losses = [-x for x in neg_log_losses]
    print("----------------------------------------------------------------------")
    print("max_depth:",depth)
    print("CV logloss",np.mean(log_losses))

----------------------------------------------------------------------
('max_depth:', 5)
('CV logloss', 1.0776594232835266)
----------------------------------------------------------------------
('max_depth:', 10)
('CV logloss', 1.0582727287365181)
----------------------------------------------------------------------
('max_depth:', 20)
('CV logloss', 1.0291053118601903)
----------------------------------------------------------------------
('max_depth:', 100)
('CV logloss', 0.9748831673859627)
----------------------------------------------------------------------
('max_depth:', 200)
('CV logloss', 0.9854028233592302)
CPU times: user 36.6 s, sys: 11.9 s, total: 48.6 s
Wall time: 2min 29s


In [32]:
# check results with a higher number of estimators
rf = RandomForestClassifier(n_estimators=1000,n_jobs=-1,max_depth=100)
neg_log_losses = cross_val_score(rf, X_train, np.ravel(y_train), cv=5, scoring="neg_log_loss")
log_losses = [-x for x in neg_log_losses]
print("CV logloss",np.mean(log_losses))

('CV logloss', 0.9504948728835766)


# final predictions

In [34]:
%%time
rf = RandomForestClassifier(n_estimators=1000,n_jobs=-1,max_depth=100)
rf.fit(X_train, np.ravel(y_train))
pred_test = rf.predict_proba(X_test)

CPU times: user 7min 15s, sys: 1.1 s, total: 7min 17s
Wall time: 2min 8s


# submit

In [35]:
df_submission = pd.DataFrame(pred_test, index=X_test_index)

In [36]:
df_submission

,0,1,2
id,,,
0,0.172846,0.164243,0.662911
1,0.318958,0.266533,0.414509
2,0.247665,0.721040,0.031295
3,0.129430,0.238071,0.632499
4,0.512244,0.289359,0.198398
5,0.473997,0.495044,0.030959
6,0.266829,0.298948,0.434222
7,0.335390,0.583538,0.081072
8,0.272147,0.197345,0.530508


In [154]:
import io, math, requests

# Ne fonctionne qu'en Python3, voir commentaire ci-dessous pour Python2
def submit_prediction(df, sep=',', **kwargs):
    # TOKEN a recuperer sur la plateforme: "Submissions" > "Submit from your Python Notebook"
    TOKEN='37760905e1b8e6e98e0c5ce188a74eae75cb3750b26031fae91aba16e5ed87f5e5138fd1c56166329d2b80a603f9a5a105287f3fc7a02d8accbfd8f9a136e862'  
    URL='https://qscore.meilleurdatascientistdefrance.com/api/submissions'
    buffer = io.BytesIO() # Python 2
    df.to_csv(buffer, sep=sep, **kwargs)
    buffer.seek(0)
    r = requests.post(URL, headers={'Authorization': 'Bearer {}'.format(TOKEN)},files={'datafile': buffer})
    if r.status_code == 429:
        raise Exception('Submissions are too close. Next submission is only allowed in {} seconds.'.format(int(math.ceil(int(r.headers['x-rate-limit-remaining']) / 1000.0))))
    if r.status_code != 200:
        raise Exception(r.text)

In [155]:
submit_prediction(df_submission, sep=',', index=True)